<a href="https://colab.research.google.com/github/saurabhsingh1411/Deep_Learning_Projects/blob/main/NLP/TOP%20PROJECTS/Toxic%20statement%20Classifier/Toxic_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [11]:
print(train.shape)

(159571, 8)


In [12]:
train['toxic'].value_counts()

0    144277
1     15294
Name: toxic, dtype: int64

In [13]:
train.isnull().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [14]:
#cleaning the dataset
import re
def process_text(text):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", "",text.lower()).split())

In [15]:
processed_text=[]
for item in range(train.shape[0]):
  processed_text.append(process_text(str(train['comment_text'][item])))

In [17]:
train['processed_comment_text']=processed_text

train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,processed_comment_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanationwhy the edits made under my usernam...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,daww he matches this background colour im seem...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man im really not trying to edit war its j...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,morei cant make any real suggestions on improv...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir are my hero any chance you remember wh...


preprocessing the text

In [21]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

max_feature=10000
tokenizer=Tokenizer(num_words=max_feature,oov_token=oov_tok)
tokenizer.fit_on_texts(train['processed_comment_text'].values)
X=tokenizer.texts_to_sequences(train['processed_comment_text'].values)
#padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

X=pad_sequences(X,maxlen=max_length, truncating=trunc_type)

In [22]:
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[classes].values

In [24]:
print(X.shape)
print(y.shape)

(159571, 120)
(159571, 6)


In [27]:
#Create model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout,GlobalMaxPooling1D
from keras.models import Model

inputLayer = Input(shape=120)
embedLayer = Embedding(input_dim=20000, output_dim=128)(inputLayer)
lstmLayer = LSTM(units=60, return_sequences=True)(embedLayer)
maxPool = GlobalMaxPooling1D()(lstmLayer)
dropOut1 = Dropout(0.1)(maxPool)
fcLayer1 = Dense(units=50, activation='relu')(dropOut1)
dropOut2 = Dropout(0.1)(fcLayer1)
fcLayer2 = Dense(units=6, activation='sigmoid')(dropOut2)

model = Model(inputs=inputLayer, outputs=fcLayer2)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 120)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 120, 128)          2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 120, 60)           45360     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 60)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0   

In [28]:
model.fit(X, y, batch_size=32, epochs=2, validation_split=0.1)

Epoch 1/2
4488/4488 [==============================] - 423s 94ms/step - loss: 0.1080 - accuracy: 0.8791 - val_loss: 0.0515 - val_accuracy: 0.9940
Epoch 2/2
4488/4488 [==============================] - 424s 95ms/step - loss: 0.0486 - accuracy: 0.9885 - val_loss: 0.0491 - val_accuracy: 0.9940


Predicting on Text values 

In [30]:
processed_text=[]
for item in range(test.shape[0]):
  processed_text.append(process_text(str(test['comment_text'][item])))

In [31]:
test['processed_comment_text']=processed_text

test.head()

,id,comment_text,processed_comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule is more succesful then youll ...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,from rfc the title is fine as it is imo
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",sources zawe ashton on lapland
3,00017563c3f7919a,":If you have a look back at the source, the in...",if you have a look back at the source the info...
4,00017695ad8997eb,I don't anonymously edit articles at all.,i dont anonymously edit articles at all


In [32]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

max_feature=10000
tokenizer=Tokenizer(num_words=max_feature,oov_token=oov_tok)
tokenizer.fit_on_texts(test['processed_comment_text'].values)
X=tokenizer.texts_to_sequences(test['processed_comment_text'].values)
#padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

X_test=pad_sequences(X,maxlen=max_length, truncating=trunc_type)

In [43]:
y_prediction=model.predict(X_test)

In [44]:
test[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]] = y_prediction

In [45]:
test.head(10)

,id,comment_text,processed_comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule is more succesful then youll ...,0.030532,1.858613e-05,0.003346,0.000046,0.004915,0.000453
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,from rfc the title is fine as it is imo,0.901624,8.316547e-02,0.725110,0.039351,0.378085,0.271135
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",sources zawe ashton on lapland,0.033062,1.813471e-04,0.009961,0.000489,0.006424,0.001521
3,00017563c3f7919a,":If you have a look back at the source, the in...",if you have a look back at the source the info...,0.978566,3.707266e-02,0.917473,0.001176,0.507567,0.019008
4,00017695ad8997eb,I don't anonymously edit articles at all.,i dont anonymously edit articles at all,0.041458,9.374566e-05,0.005572,0.000365,0.006021,0.001035
5,0001ea8717f6de06,Thank you for understanding. I think very high...,thank you for understanding i think very highl...,0.997838,3.903538e-01,0.978127,0.015752,0.829809,0.113645
6,00024115d4cbde0f,Please do not add nonsense to Wikipedia. Such ...,please do not add nonsense to wikipedia such e...,0.985064,7.278538e-02,0.925427,0.004058,0.619059,0.047745
7,000247e83dcc1211,:Dear god this site is horrible.,dear god this site is horrible,0.002270,2.586656e-06,0.000307,0.000015,0.000265,0.000041
8,00025358d4737918,""" \n Only a fool can believe in such numbers. ...",only a fool can believe in such numbers the co...,0.000867,4.637344e-07,0.000188,0.000002,0.000108,0.000017
9,00026d1092fe71cc,== Double Redirects == \n\n When fixing double...,double redirects when fixing double redirects ...,0.002884,3.334319e-06,0.000522,0.000010,0.000425,0.000072


In [ ]:
test[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]] = y_prediction

In [47]:
for i in range(test.shape[0]):
  if test['toxic'][i]<0.5:
    test['toxic'][i]=0
  else :
    test['toxic'][i]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [48]:
test.head()

,id,comment_text,processed_comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule is more succesful then youll ...,0.0,0.000019,0.003346,0.000046,0.004915,0.000453
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,from rfc the title is fine as it is imo,1.0,0.083165,0.725110,0.039351,0.378085,0.271135
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",sources zawe ashton on lapland,0.0,0.000181,0.009961,0.000489,0.006424,0.001521
3,00017563c3f7919a,":If you have a look back at the source, the in...",if you have a look back at the source the info...,1.0,0.037073,0.917473,0.001176,0.507567,0.019008
4,00017695ad8997eb,I don't anonymously edit articles at all.,i dont anonymously edit articles at all,0.0,0.000094,0.005572,0.000365,0.006021,0.001035


In [49]:
#severe_toxic
for i in range(test.shape[0]):
  if test['severe_toxic'][i]<0.5:
    test['severe_toxic'][i]=0
  else :
    test['severe_toxic'][i]=1
#obscene	
for i in range(test.shape[0]):
  if test['obscene'][i]<0.5:
    test['obscene'][i]=0
  else :
    test['obscene'][i]=1
#threat
for i in range(test.shape[0]):
  if test['threat'][i]<0.5:
    test['threat'][i]=0
  else :
    test['threat'][i]=1
#insult
for i in range(test.shape[0]):
  if test['insult'][i]<0.5:
    test['insult'][i]=0
  else :
    test['insult'][i]=1
#identity_hate
for i in range(test.shape[0]):
  if test['identity_hate'][i]<0.5:
    test['identity_hate'][i]=0
  else :
    test['identity_hate'][i]=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/py

In [50]:
test.head()

,id,comment_text,processed_comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,yo bitch ja rule is more succesful then youll ...,0.0,0.0,0.0,0.0,0.0,0.0
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,from rfc the title is fine as it is imo,1.0,0.0,1.0,0.0,0.0,0.0
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",sources zawe ashton on lapland,0.0,0.0,0.0,0.0,0.0,0.0
3,00017563c3f7919a,":If you have a look back at the source, the in...",if you have a look back at the source the info...,1.0,0.0,1.0,0.0,1.0,0.0
4,00017695ad8997eb,I don't anonymously edit articles at all.,i dont anonymously edit articles at all,0.0,0.0,0.0,0.0,0.0,0.0
